<div style="background-color:#000;"><img src="pqn.png"></img></div>

This notebook demonstrates how to backtest a multi-timeframe (MTF) trading strategy using VectorBT PRO. The code involves fetching asset price data, calculating technical indicators across multiple timeframes, and setting up a unified trading portfolio. It addresses common pitfalls in MTF analysis, such as look-ahead bias and information loss, by resampling data effectively. The final output includes visualizations of the cumulative returns and trade signals for each timeframe, aiding in financial modeling and strategy optimization.

In [ ]:
from vectorbtpro import *

Configure our graphs to be dark and gap-free.

In [ ]:
vbt.settings.set_theme("dark")
vbt.settings.plotting.auto_rangebreaks = True

Fetch hourly price data for TSLA from 2023 to 2024.

In [ ]:
data = vbt.YFData.pull("TSLA", start="2023", end="2024", timeframe="hourly")

Calculate the fast and slow SMA indicators across different timeframes.

In [ ]:
fast_sma = data.run(
    "talib:sma", 
    timeframe=["1h", "4h", "1d"], 
    timeperiod=vbt.Default(20),
    skipna=True
)

In [ ]:
slow_sma = data.run(
    "talib:sma", 
    timeframe=["1h", "4h", "1d"], 
    timeperiod=vbt.Default(50),
    skipna=True
)

Plot the fast SMA indicators to observe their behavior across timeframes.

In [ ]:
fast_sma.real.vbt.plot().show_png()

Set up a trading portfolio based on SMA crossover signals and allocate capital.

In [ ]:
pf = vbt.PF.from_signals(
    data, 
    long_entries=fast_sma.real_crossed_above(slow_sma), 
    short_entries=fast_sma.real_crossed_below(slow_sma), 
    size=[[0.05, 0.1, 0.2]],
    size_type="valuepercent",
    init_cash=10_000,
    group_by=["pf", "pf", "pf"],
    cash_sharing=True,
    tsl_stop=0.2
)

Plot cumulative returns for each timeframe and the entire portfolio.

In [ ]:
fig = (
    pf
    .get_cumulative_returns()
    .vbt
    .plot(trace_kwargs=dict(line_color="gray", line_dash="dot"))
)

In [ ]:
fig = pf.get_cumulative_returns(group_by=False).vbt.plot(fig=fig)
fig.show_png()

Plot indicators and trade signals for the daily timeframe.

In [ ]:
fig = fast_sma.real.vbt.plot(column="1d", trace_kwargs=dict(name="Fast", line_color="limegreen"))
fig = slow_sma.real.vbt.plot(column="1d", trace_kwargs=dict(name="Slow", line_color="orangered"), fig=fig)
fig = pf.plot_trade_signals(column="1d", fig=fig)
fig.show_png()

Combine indicators from different timeframes and identify the best performing pairs.

In [ ]:
fast_sma_real = fast_sma.real.vbt.rename_levels({"sma_timeframe": "fast_sma_timeframe"})
slow_sma_real = slow_sma.real.vbt.rename_levels({"sma_timeframe": "slow_sma_timeframe"})
fast_sma_real, slow_sma_real = fast_sma_real.vbt.x(slow_sma_real)
long_entries = fast_sma_real.vbt.crossed_above(slow_sma_real)
short_entries = fast_sma_real.vbt.crossed_below(slow_sma_real)
pf = vbt.PF.from_signals(data, long_entries=long_entries, short_entries=short_entries)
pf.trades.expectancy.sort_values(ascending=False)

Timeframe is another strategy parameter to tweak for novel insights and cross-validation.

Timeframes offer a unique dimension for strategy optimization and should be cross-validated like other parameters.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advise. Use at your own risk.